# Определение пар товаров, которые покупают вместе

Пары товаров подобраны по ассоциативным правилам (Association Rules) с помощью алгоритма apriori, реализованного в библиотеке MLxtend для Python (http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/)

### Подключение библиотек, подгрузка, группировка, выборка данных для модели

In [5]:
# подключаем библиотеки numpy, pandas, mlxtend

import pandas as pd
import numpy as np

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori as apriori
from mlxtend.frequent_patterns import association_rules

In [6]:
# подгружаем данныые, смотрим первые 5 строк
data = pd.read_csv('dataset_for_test_20200729_cleaned.csv',sep=';', decimal=",")
data.head()

,order_id,_date,user_id,plu_id,category_id,parent_category_id,product_quantity,product_sum
0,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,3383918,855.0,1105.0,1.0,49.9
1,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,3489545,853.0,849.0,1.0,89.9
2,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,2068955,839.0,834.0,1.0,99.9
3,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,3435295,838.0,834.0,2.0,33.8
4,5efc2687f59a7f82441fe4ca,2020-07-01 09:00:39+03,5efb12173849bd7ecc4cdb50,3435297,838.0,834.0,2.0,31.8


In [11]:
# проверяем размерность таблицы
data.shape

(332642, 8)

In [8]:
# делаем выборку только двух полей для модели: id заказа, id товара; смотрим первые 5 строк
df = data.loc[:, ['order_id', 'plu_id']]
df.head()

,order_id,plu_id
0,5efc2687f59a7f82441fe4ca,3383918
1,5efc2687f59a7f82441fe4ca,3489545
2,5efc2687f59a7f82441fe4ca,2068955
3,5efc2687f59a7f82441fe4ca,3435295
4,5efc2687f59a7f82441fe4ca,3435297


In [9]:
# проверяем размерность данных выборки, на всякий случай
df.shape

(332642, 2)

In [12]:
# группируем плоские данные по заказам, продукты в одном заказе схлопываются в список в одной строке
df2 = df.groupby('order_id').agg(lambda x: x.tolist())
df2.head()

,plu_id
order_id,
5efc2687f59a7f82441fe4ca,"[3383918, 3489545, 2068955, 3435295, 3435297, ..."
5efc26a0d9c459f11a8e122f,"[1860, 2139858, 3170135, 3341462, 45463, 36955..."
5efc26a4d9c45903d38e1247,"[2137027, 1536, 3661004, 3664384, 3674166, 368..."
5efc26b4d9c459748e8e126c,"[3477505, 43693, 79108, 3639749, 3444750, 3682..."
5efc26c1f59a7f62941fe522,"[2144368, 18510, 3480546, 3224596, 3998557, 40..."


In [13]:
# смотрим размерность полученного агрегата
df2.shape

(23773, 1)

In [15]:
# датасет типа pandas data frame преобразовываем в обычный список для модели, которая на вход принимает объект типа список,
# и снова проверяю его длину
temp = df2['plu_id'].to_list()
len(temp)

23773

### Создание модели для поиска ассоциативных правил

In [23]:
# создаем объект модели, передаем в нее данные, обучаем
te = TransactionEncoder()
te_ary = te.fit(temp).transform(temp)
df = pd.DataFrame(te_ary, columns=te.columns_)

### В следующем участке кода определяются значения нескольких гиперпараметров для модели. Управляя ими, мы определяем результат ее работы. Условно, если задать высокие требования к модели,то на выходе получим мало пар (или совсем не получим). В данном экземпляре конкретные значения гиперпараметров найдены эмпирическим способом. 

------

Тут задаем уровень минимальной поддержки - фактически, это минимальная доля заказов в датасете, в которых есть пары товаров. 

In [28]:
# На датасете примера высокие значения этого параметра приводили к нулевому результату, возможно, из разношерстности заказов по
# товарным наборам. В итоге, чтобы модель заработала и дала выхлоп, остановился на таком значении:
MIN_SUPPORT = 0.03

frequent_itemsets = apriori(df, min_support=MIN_SUPPORT, use_colnames=True)
# frequent_itemsets


### Далее из полученного на пред.шаге набора правил отбираем пары с заданным уровнем качества. Пары фильтруются 2 раза, независимо друг от друга по приницпу - давайте отберем по одному признаку или давайте по другому


---

Тут отбираем по Confidence. Он о том, как часто это правило работает для датасета (процент тех, кто покупает оба товара в проценте от тех, кто купил первый). 

In [42]:
# в данном случае эмипирически выбрано следующее минимальное значение: 
CONFIDENCE = 0.1

# запуск фильтрации
pairs_conf = association_rules(frequent_itemsets, metric="confidence", min_threshold=CONFIDENCE)

# вывод результата работы модели со статистическими показателями, выборка нескольких строк
pairs_conf.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(4072),(5013),0.179027,0.120052,0.033357,0.186325,1.552035,0.011865,1.081449
1,(5013),(4072),0.120052,0.179027,0.033357,0.277856,1.552035,0.011865,1.136855
2,(4072),(45463),0.179027,0.105203,0.033820,0.188910,1.795663,0.014986,1.103202
3,(45463),(4072),0.105203,0.179027,0.033820,0.321471,1.795663,0.014986,1.209932
4,(4072),(45505),0.179027,0.165398,0.057965,0.323778,1.957574,0.028354,1.234214


В списке выше идшки товаров приведены в полях antecedents и consequents.

In [46]:
# делаем выборку полей только с идшками пар товаров
pairs_conf_data = pairs.loc[:, ['antecedents', 'consequents']]
pairs_conf_data

,antecedents,consequents
0,(4072),(5013)
1,(5013),(4072)
2,(4072),(45463)
3,(45463),(4072)
4,(4072),(45505)
5,(45505),(4072)
6,(4072),(3638820)
7,(3638820),(4072)
8,(4072),(3639749)
9,(3639749),(4072)


------

Тут отбираем пары по Lift. Он о достоверности правила (отношение: оба товара покупаются вместе / товар покупался вообще). Значение д.б. больше 1 для отбора работающих правил.

In [47]:
# фильтруем по минимальном значению 1.2
LIFT = 1.2

rules_lift = association_rules(frequent_itemsets, metric="lift", min_threshold=LIFT)
rules_lift_data = rules_lift.loc[:, ['antecedents', 'consequents']]
rules_lift_data

,antecedents,consequents
0,(4072),(5013)
1,(5013),(4072)
2,(4072),(45463)
3,(45463),(4072)
4,(4072),(45505)
5,(45505),(4072)
6,(4072),(3638820)
7,(3638820),(4072)
8,(4072),(3639749)
9,(3639749),(4072)
